# Imports

---



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

ImportError: No module named 'keras.applications.resnet'

# Make Model

---



In [ ]:
def make_model():
    base_model = ResNet50(include_top=False, input_shape=(150, 150, 3), weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x) 
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x) 
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x) 
    x = Dropout(0.3)(x)
    predictions = Dense(6, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
make_model().summary()

# Setting up Input Data Generators

---



In [ ]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')
    plt.savefig('acc_vs_epochs.png')

In [ ]:
train_path = 'data/train/'

In [ ]:
valid_path = 'data/valid/'

No data augmentation for now.

In [ ]:
train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=20,
      horizontal_flip=True,
      vertical_flip=False
)

In [ ]:
valid_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=20,
      horizontal_flip=True,
      vertical_flip=False
)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path, target_size=(150, 150), 
                                                    batch_size=32, shuffle=True, seed=42)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=(150, 150), 
                                                    batch_size=32, shuffle=True, seed=42)

# Training

---

In [ ]:
model = make_model()

In [ ]:
model.compile(Adam(lr=0.0001, decay=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
filepath="./checkpoints/" + "VGG16" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
earlystop = EarlyStopping(monitor='val_loss', patience=1)
callbacks_list = [checkpoint]

In [ ]:
history = model.fit_generator(train_generator, 
                              epochs = 10, 
                              steps_per_epoch = 13627 // 32,
                              shuffle = True, 
                              validation_data = valid_generator,
                              validation_steps = 3407 // 32,
                              callbacks = callbacks_list)

In [ ]:
plot_training(history)

In [ ]:
import os

In [ ]:
test_path = 'data/test'

In [ ]:
results = []

In [ ]:
for image_name in os.listdir(test_path):
    image_path = test_path + '/' + image_name
    image = load_img(image_path, target_size=(150, 150))
    image = img_to_array(image)
    image = np.array(image)
    image = preprocess_input(image)
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    prediction = np.argmax(model.predict(image))
    results.append({'image_name': image_name, 'label':prediction})

In [ ]:
df = pd.DataFrame(results, columns=['image_name', 'label'])

In [ ]:
df['label'].value_counts()

In [ ]:
df.to_csv("resnet_big_10epochs_higherlr.csv", index=False)